In [ ]:
!pip install langchain langchain_openai langchain-community wikipedia tiktoken openai chromadb faiss-cpu

In [ ]:
# Install OpenAI if not installed
!pip install openai

# Load key securely without displaying it
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")


In [ ]:
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

In [ ]:
documents = [
    Document(
        page_content="Climate change is a long-term alteration of temperature and typical weather patterns in a place.",
        metadata={"topic": "Climate Change"}
    ),
    Document(
        page_content="AI enables computers and machines to mimic human intelligence and perform tasks like problem-solving.",
        metadata={"topic": "Artificial Intelligence"}
    ),
    Document(
        page_content="The Great Wall of China was built to protect Chinese states against invasions and raids from nomadic groups.",
        metadata={"topic": "The Great Wall of China"}
    ),
    Document(
        page_content="Electric vehicles use electric motors powered by batteries, reducing carbon emissions significantly.",
        metadata={"topic": "Electric Vehicles"}
    ),
    Document(
        page_content="The human brain is a complex organ responsible for thought, memory, emotion, and sensory processing.",
        metadata={"topic": "The Human Brain"}
    ),
    Document(
        page_content="Paris is the capital of France. Photosynthesis is the process used by plants to convert light energy into chemical energy. Biology is an important subject.",
        metadata={"topic": "Photosynthesis"}
    ),
    Document(
        page_content="Space exploration involves investigating outer space using astronomy, space technology, and spacecraft.",
        metadata={"topic": "Space Exploration"}
    ),
    Document(
        page_content="COVID-19 affected global health systems and economies, highlighting the need for preparedness and vaccines.",
        metadata={"topic": "COVID-19 Pandemic"}
    ),
    Document(
        page_content="Ancient Egypt was a civilization known for pyramids, hieroglyphics, and a deep belief in the afterlife.",
        metadata={"topic": "Ancient Egypt"}
    ),
    Document(
        page_content="Blockchain is a decentralized ledger system that records transactions in a secure and transparent way.",
        metadata={"topic": "Blockchain Technology"}
    ),
]

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
from langchain_community.vectorstores import FAISS

In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [ ]:
from langchain.retrievers.document_compressors import LLMChainExtractor

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever

**Contextual Compression Retriever**

In [ ]:
vs = FAISS.from_documents(
    documents = documents,
    embedding=OpenAIEmbeddings()
)

In [ ]:
base_retriever = vs.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

In [ ]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

In [ ]:
result_compression = compression_retriever.invoke("What is Photosynthesis?")

In [ ]:
for i, doc in enumerate(result_compression):
    print(f"{i+1}. {doc.page_content}")

**Multi Query Retriever**

In [ ]:
vector_store_faiss = FAISS.from_documents(
    documents = documents,
    embedding=OpenAIEmbeddings()
)

In [ ]:
retriever_similarity = vector_store_faiss.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [ ]:
retriever_multi = MultiQueryRetriever.from_llm(
    retriever=retriever_similarity,
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
)

In [ ]:
query = "What protects environment?"

In [ ]:
result_similarity = retriever_similarity.invoke(query)

In [ ]:
result_multi = retriever_multi.invoke(query)

In [ ]:
for i, doc in enumerate(result_similarity):
    print(f"{i+1}. {doc.page_content}")

In [ ]:
for i, doc in enumerate(result_multi):
    print(f"{i+1}. {doc.page_content}")

**MMR**

In [ ]:
retriever_mmr = vector_store_faiss.as_retriever(search_type="mmr", search_kwargs={"k": 2, "lambda_mult":0.9})

In [ ]:
query = 'Which docs talks about Biology?'

In [ ]:
result_faiss = retriever_mmr.invoke(query)

In [ ]:
for i, doc in enumerate(result_faiss):
    print(f"{i+1}. {doc.page_content}")

**Vector Store Retriever**

In [ ]:
vector_store = Chroma.from_documents(
    documents = documents,
    embedding=OpenAIEmbeddings(),
    collection_name="my_collection"
)

In [ ]:
retriever_vs = vector_store.as_retriever(search_kwargs={"k": 2})

In [ ]:
query='Why EVs are important?'

In [ ]:
result = retriever_vs.invoke(query)

In [ ]:
for i, doc in enumerate(result):
    print(f"{i+1}. {doc.page_content}")

**Wikipedia Retriever**

In [ ]:
from langchain_community.retrievers import WikipediaRetriever
retriever_wk = WikipediaRetriever(top_n_results=2, lang='en')

In [ ]:
query = 'Badr Wars non-existence in non-islamic literature'

In [ ]:
docs = retriever_wk.invoke(query)

In [ ]:
for i, doc in enumerate(docs):
    print(f"{i+1}. {doc.page_content}")